In [1]:
import pandas as pd

In [2]:
mult_reg_results, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Video/correlation_hamming_regression_results.pkl')

In [3]:
mult_reg_results

,experiment_type,stimuli_type,electrode_region,model_name,time,beta_value,t_value,p_value,lower_conf_interval,upper_conf_interval
513,naive,video,central,action_category_hamming,-200,11.316784,1.238944,0.216458,-6.667776,29.301344
4117,naive,video,frontal,flowModel_10_frames_correlation,-200,-0.016759,-0.590434,0.555401,-0.072646,0.039128
4116,naive,video,frontal,biological_appearance_hamming,-200,0.010268,0.169731,0.865350,-0.108848,0.129385
4115,naive,video,frontal,agent_hamming,-200,0.036869,1.583123,0.114582,-0.008985,0.082722
4114,naive,video,frontal,action_target_hamming,-200,-0.293953,-2.244120,0.025647,-0.551858,-0.036048
...,...,...,...,...,...,...,...,...,...,...
3397,naive,video,central,flowModel_10_frames_correlation,598,-0.034803,-1.060776,0.289754,-0.099400,0.029795
3398,naive,video,central,movement_nature_hamming,598,0.380521,1.110887,0.267620,-0.293909,1.054951
3399,naive,video,central,gabor_video_10_frames_correlation,598,-0.035188,-1.224890,0.221700,-0.091750,0.021374
6997,naive,video,frontal,flowModel_10_frames_correlation,598,0.004941,0.146456,0.883672,-0.061487,0.071370


In [4]:
VIF =  pd.read_csv('/auto/data2/oelmas/RSA_Results/Naive/Video/_VIF_Results.csv')

In [5]:
VIF

,Unnamed: 0,VIF Factor,model_name
0,0,1.233069,action_category_hamming
1,1,1.180457,action_target_hamming
2,2,4.987882,agent_hamming
3,3,2.901776,biological_appearance_hamming
4,4,3.545342,flowModel_10_frames_correlation
5,5,2.224759,movement_nature_hamming
6,6,3.863748,gabor_video_10_frames_correlation
7,7,1.809381,avg_pixel_intensity_model_hamming
8,8,1.641199,pixel_wise_motion_model_hamming


In [6]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [7]:
sig_idx = multipletests(mult_reg_results["p_value"], alpha=0.05, method='bonferroni')[0]
mult_reg_results['significant'] = sig_idx

In [ ]:
#er_idx = mult_reg_results.electrode_region=="parietal" 
#mod_idx = mult_reg_results.model_name=="movement_nature_hamming" 
#df = mult_reg_results[er_idx & mod_idx]

#df[df.beta_value == df.beta_value.max()]

In [8]:
mult_reg_results

,experiment_type,stimuli_type,electrode_region,model_name,time,beta_value,t_value,p_value,lower_conf_interval,upper_conf_interval,significant
513,naive,video,central,action_category_hamming,-200,11.316784,1.238944,0.216458,-6.667776,29.301344,False
4117,naive,video,frontal,flowModel_10_frames_correlation,-200,-0.016759,-0.590434,0.555401,-0.072646,0.039128,False
4116,naive,video,frontal,biological_appearance_hamming,-200,0.010268,0.169731,0.865350,-0.108848,0.129385,False
4115,naive,video,frontal,agent_hamming,-200,0.036869,1.583123,0.114582,-0.008985,0.082722,False
4114,naive,video,frontal,action_target_hamming,-200,-0.293953,-2.244120,0.025647,-0.551858,-0.036048,False
...,...,...,...,...,...,...,...,...,...,...,...
3397,naive,video,central,flowModel_10_frames_correlation,598,-0.034803,-1.060776,0.289754,-0.099400,0.029795,False
3398,naive,video,central,movement_nature_hamming,598,0.380521,1.110887,0.267620,-0.293909,1.054951,False
3399,naive,video,central,gabor_video_10_frames_correlation,598,-0.035188,-1.224890,0.221700,-0.091750,0.021374,False
6997,naive,video,frontal,flowModel_10_frames_correlation,598,0.004941,0.146456,0.883672,-0.061487,0.071370,False


In [9]:
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as po

In [10]:
avg_sbj_results, eeg_dist, model_dist  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Video/correlation_hamming_avg_subjects_results.pkl')

In [11]:
rdm_df = avg_sbj_results
model_name = 'biological_appearance_hamming'
electrode_region = 'parietal'
constant = 'naive'
variable = 'video'

In [12]:
df_to_plot = rdm_df[  (rdm_df["model_name"] == model_name) 
                    & (rdm_df["electrode_region"] == electrode_region)
                    & (rdm_df["experiment_type"] == constant) 
                    & (rdm_df["stimulus_type"] == variable)]

In [13]:
mult_reg_results.loc[mult_reg_results['significant']].model_name.unique()

array(['gabor_video_10_frames_correlation',
       'avg_pixel_intensity_model_hamming',
       'flowModel_10_frames_correlation', 'biological_appearance_hamming'],
      dtype=object)

In [18]:
(mult_reg_results.loc[mult_reg_results['significant']])


,experiment_type,stimuli_type,electrode_region,model_name,time,beta_value,t_value,p_value,lower_conf_interval,upper_conf_interval,significant
10635,naive,video,parietal,gabor_video_10_frames_correlation,64,0.116709,5.268061,2.849686e-07,0.073089,0.160328,True
10644,naive,video,parietal,gabor_video_10_frames_correlation,66,0.129551,5.882500,1.210166e-08,0.086189,0.172913,True
21454,naive,video,occipital,avg_pixel_intensity_model_hamming,68,0.708393,4.881884,1.814254e-06,0.422690,0.994097,True
10653,naive,video,parietal,gabor_video_10_frames_correlation,68,0.126287,5.782956,2.053990e-08,0.083290,0.169284,True
10662,naive,video,parietal,gabor_video_10_frames_correlation,70,0.112507,5.272789,2.784010e-07,0.070495,0.154518,True
...,...,...,...,...,...,...,...,...,...,...,...
19066,naive,video,occipital,flowModel_10_frames_correlation,130,0.202270,6.148322,2.855086e-09,0.137496,0.267045,True
19663,naive,video,occipital,avg_pixel_intensity_model_hamming,248,0.544951,4.880895,1.822626e-06,0.325121,0.764781,True
19672,naive,video,occipital,avg_pixel_intensity_model_hamming,250,0.612136,5.429486,1.274049e-07,0.390154,0.834117,True
19681,naive,video,occipital,avg_pixel_intensity_model_hamming,252,0.617764,5.450267,1.147118e-07,0.394596,0.840933,True


In [ ]:
er_idx = mult_reg_results.electrode_region=="parietal" 
mod_idx = mult_reg_results.model_name=="biological_appearance_hamming" 
reg_df = mult_reg_results[er_idx & mod_idx]

In [ ]:
reg_df.loc[reg_df['significant']].time.tolist()

In [ ]:
fig = go.Figure()
 
# Plots the kendall tau curve
fig.add_trace(go.Scatter(x=df_to_plot["time"],
                        y=df_to_plot["kendall_tau"],
                        mode='lines'))


fig.add_shape(
        # Rectangle reference to the plot
            type="rect",
            xref="x",
            yref="paper",
            x0=min(reg_df.loc[reg_df['significant']].time),
            y0=0,
            x1=max(reg_df.loc[reg_df['significant']].time),
            y1=1,
            opacity=0.7,
            layer="below",
            line_width=0,
            fillcolor="LightSalmon",
        )

fig.update_yaxes(range=[-0.2, 0.6])
fig.update_yaxes(title_text="Kendall-tau")
fig.update_xaxes(title_text="Time ms (relative to stimulus onset)")

po.plot(fig, filename = "outputs/TEST.html", auto_open=False)

In [ ]:
mult_reg_results.model_name.unique()

In [ ]:
#############################################################
#############################################################


In [23]:
er_idx = mult_reg_results.electrode_region=="occipital" 
ex_idx = mult_reg_results.experiment_type=='naive'
st_idx = mult_reg_results.stimuli_type=='video'

for model in mult_reg_results.model_name.unique():
    mod_idx = mult_reg_results.model_name==model
    
    reg_df = mult_reg_results[ex_idx & st_idx & er_idx & mod_idx]
    sig_idx = multipletests(reg_df["p_value"], alpha=0.05, method='bonferroni')[0]
    sig_time = reg_df[sig_idx].time



In [24]:
sig_idx = multipletests(mult_reg_results["p_value"], alpha=0.05, method='bonferroni')[0]
mult_reg_results['significant'] = sig_idx

In [26]:
reg_df = mult_reg_results[ex_idx & st_idx & er_idx]


fig = px.scatter(reg_df.loc[reg_df['significant']], 
                 x="time", y="model_name",  color="model_name",
                title='Significance comparison of Regressors')

#fig.update_yaxes(title_text="Models")
fig.update_xaxes(range=[-0.2, 0.6])

#fig.update_xaxes(title_text="Time ms (relative to stimulus onset)")


po.plot(fig, filename = "outputs/SCATTER2.html", auto_open=False)

'outputs/SCATTER2.html'